In [2]:
import sys

sys.path.append("/home/xmh/DeepCTR-Torch")

# -*- coding: utf-8 -*-
import pandas as pd
import torch
from sklearn.metrics import log_loss, roc_auc_score,accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
from deepctr_torch.models import *

import numpy as np

ModuleNotFoundError: No module named 'torch'

In [2]:
import os
if os.path.exists("/data/web_server/project/avazu-ctr-prediction/"):
    data_prefix = "/data/web_server/project/avazu-ctr-prediction/"
else:
    data_prefix = "/data/avazu-ctr-prediction/"
# df = pd.read_csv(data_prefix + "train-mini")
df = pd.read_csv(data_prefix + "train")

In [3]:
data = df
sparse_features = ['id', 'C1', 'banner_pos', 'site_id', 'site_domain',
                   'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
                   'device_ip', 'device_model', 'device_type', 'device_conn_type', ] \
                  + ['C' + str(i) for i in range(14, 22)]
dense_features = []

target = ['click']

# 1.Label Encoding for sparse features,and do simple Transformation for dense features

mms = MinMaxScaler(feature_range=(0, 1))
if dense_features != []:
    data[dense_features] = mms.fit_transform(data[dense_features])

counts = []
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
    counts.append(len(lbe.classes_))
counts

[40428967,
 7,
 7,
 4737,
 7745,
 26,
 8552,
 559,
 36,
 2686408,
 6729486,
 8251,
 5,
 4,
 2626,
 8,
 9,
 435,
 4,
 68,
 172,
 60]

In [4]:
print(counts)

[40428967, 7, 7, 4737, 7745, 26, 8552, 559, 36, 2686408, 6729486, 8251, 5, 4, 2626, 8, 9, 435, 4, 68, 172, 60]


In [5]:
data

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,2192468,0,14102100,2,0,582,7339,2,7884,254,...,1,1,285,3,2,56,0,2,0,22
1,21916663,0,14102100,2,0,582,7339,2,7884,254,...,1,0,283,3,2,56,0,2,65,22
2,21917117,0,14102100,2,0,582,7339,2,7884,254,...,1,0,283,3,2,56,0,2,65,22
3,21917710,0,14102100,2,0,582,7339,2,7884,254,...,1,0,285,3,2,56,0,2,65,22
4,21917799,0,14102100,2,1,4695,4456,0,7884,254,...,1,0,590,3,2,107,0,2,0,42
5,21917885,0,14102100,2,0,3948,5717,24,7884,254,...,1,0,336,3,2,75,0,27,59,39
6,21917896,0,14102100,2,0,2678,1193,24,7884,254,...,1,0,739,3,2,149,0,4,0,42
7,21918323,0,14102100,2,1,4158,3893,24,7884,254,...,1,0,777,3,2,159,3,4,0,6
8,21918397,1,14102100,2,0,582,7339,2,7884,254,...,1,1,286,3,2,56,0,2,0,22
9,21919127,0,14102100,1,0,2474,6000,6,7884,254,...,0,0,1017,3,2,210,3,11,145,6


In [6]:
data.to_csv("/home/xmh/avazu-train.csv", index=False)

In [4]:


# 2.count #unique features for each sparse field,and record dense feature field name

fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].nunique(), embedding_dim=4)
                          for i, feat in enumerate(sparse_features)] + [DenseFeat(feat, 1,)
                                                                        for feat in dense_features]

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

# 3.generate input data for modelf

train, test = train_test_split(data, test_size=0.1)
train_model_input = {name: train[name] for name in feature_names}
test_model_input = {name: test[name] for name in feature_names}

In [6]:
# 4.Define Model,train,predict and evaluate
import argparse
import numpy as np
import random
seed = 100
torch.manual_seed(seed) # cpu
torch.cuda.manual_seed(seed) #gpu
np.random.seed(seed) #numpy
random.seed(seed) #random and transforms
torch.backends.cudnn.deterministic=True # cudn


parser = argparse.ArgumentParser()
parser.add_argument("--model", choices=('deepfm',
                                        'wdl', 'dcn'),
                    default='deepfm')

parser.add_argument("--dense-opt", choices=('adam',
                                            'sgd', 'adagrad', 'adadelta'), default='sgd')
parser.add_argument("--sparse-opt", choices=('adam',
                                             'sgd', 'adagrad', 'adadelta', 'rrms'), default='adam')
# sgd,rrms,adadelta 有问题


parser.add_argument("--dense-lr", type=float, default=0.1)
parser.add_argument("--sparse-lr", type=float, default=0.1)


args = parser.parse_args(args=[])


model_name = args.model
optimizer_dense = args.dense_opt
optimizer_sparse = args.sparse_opt
optimizer_dense_lr = args.dense_lr
optimizer_sparse_lr = args.sparse_lr

device = 'cpu'
use_cuda = True
if use_cuda and torch.cuda.is_available():
    print('cuda ready...')
    device = 'cuda:0'

if model_name == "deepfm":
    model = DeepFM(linear_feature_columns, dnn_feature_columns,
                   task='binary', device=device)
# elif model_name == "din":
#     model = DIN(linear_feature_columns, dnn_feature_columns,
#                 task='binary', device=device)
elif model_name == "wdl":
    model = WDL(linear_feature_columns, dnn_feature_columns,
                task='binary', device=device)
elif model_name == "dcn":
    model = DCN(linear_feature_columns, dnn_feature_columns,
                task='binary', device=device)

model.compile(optimizer=optimizer_dense, loss="binary_crossentropy",
              metrics=['binary_crossentropy', 'acc', 'AUC'],
              optimizer_sparse=optimizer_sparse,
              opt_lr=optimizer_dense_lr,
              opt_s_lr=optimizer_sparse_lr, )
history = model.fit(train_model_input, train[target].values,
                    batch_size=128, epochs=1, verbose=1, validation_split=0.1, model_name=model_name,
                   verbose_steps= 3)
pred_ans = model.predict(test_model_input, batch_size=128)
print("test LogLoss", round(log_loss(test[target].values, pred_ans), 4))
print("test Accuracy", round(accuracy_score(
    test[target].values, pred_ans > 0.5), 4))
print("test AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

2it [00:00, 18.50it/s]

out.bias
dnn.linears.0.weight
dnn.linears.0.bias
dnn.linears.1.weight
dnn.linears.1.bias
dnn_linear.weight
embedding_dict.C1.weight
embedding_dict.C14.weight
embedding_dict.C15.weight
embedding_dict.C16.weight
embedding_dict.C17.weight
embedding_dict.C18.weight
embedding_dict.C19.weight
embedding_dict.C20.weight
embedding_dict.C21.weight
embedding_dict.app_category.weight
embedding_dict.app_domain.weight
embedding_dict.app_id.weight
embedding_dict.banner_pos.weight
embedding_dict.device_conn_type.weight
embedding_dict.device_id.weight
embedding_dict.device_ip.weight
embedding_dict.device_model.weight
embedding_dict.device_type.weight
embedding_dict.id.weight
embedding_dict.site_category.weight
embedding_dict.site_domain.weight
embedding_dict.site_id.weight
linear_model.embedding_dict.C1.weight
linear_model.embedding_dict.C14.weight
linear_model.embedding_dict.C15.weight
linear_model.embedding_dict.C16.weight
linear_model.embedding_dict.C17.weight
linear_model.embedding_dict.C18.weight


3it [00:00,  7.00it/s]

[Iter2] - loss:  5.7591 - binary_crossentropy:  2.1197 - acc:  0.8255 - auc:  0.5137 - val_binary_crossentropy:  2.8475 - val_acc:  0.8233 - val_auc:  0.5230


6it [00:00,  7.01it/s]

...

66it [00:09,  7.16it/s]

[Iter65] - loss:  15.7050 - binary_crossentropy:  8.3630 - acc:  0.4740 - auc:  0.5006 - val_binary_crossentropy:  13.1259 - val_acc:  0.1767 - val_auc:  0.5000


71it [00:09,  7.22it/s]

[Iter68] - loss:  16.0201 - binary_crossentropy:  8.5752 - acc:  0.4607 - auc:  0.5006 - val_binary_crossentropy:  13.1259 - val_acc:  0.1767 - val_auc:  0.5000


KeyboardInterrupt: 

In [ ]:
print("a")